In [1]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize
import shapely.geometry



In [2]:
sidewalks_ramp_score = pd.read_csv('../data/merged_sidewalks_and_ramps.csv')
sidewalks_hazard_score = pd.read_csv('../data/merged_sidewalks_and_hazards.csv')

In [3]:
sidewalks_hazard_score

,Unnamed: 0,GlobalID,geometry,geometryx,ID,Type
0,0,1f823362-3158-4962-a052-63c0423f0920,POINT (-5523706.675777031 9518645.961485378),POLYGON ((-5523714.355554549 9518648.746939978...,22960,Trip hazards not due to tree roots
1,1,ff26eb97-df6d-4397-818d-039286455d6e,POINT (-5523703.679574358 9518646.881700885),POLYGON ((-5523714.355554549 9518648.746939978...,22960,Trip hazards not due to tree roots
2,2,08a1879f-4d56-4f87-9bff-56a6f2afcce1,POINT (-5523693.913145971 9518643.739608342),POLYGON ((-5523714.355554549 9518648.746939978...,22960,Trip hazards not due to tree roots
3,5,0abbc4a2-cb4a-493a-8e0d-71cb6c67114b,POINT (-5524110.327023729 9518366.360779744),POLYGON ((-5524212.797930317 9518255.546939176...,3829,Trip hazards not due to tree roots
4,6,55c62d4c-ea0b-49da-9170-b657bf2a0c69,POINT (-5523944.023634829 9517716.612699214),POLYGON ((-5523951.383221917 9517714.426915333...,23066,Trip hazards not due to tree roots
...,...,...,...,...,...,...
6568,6988,28dda62b-00f9-4d0d-a3b5-2f9b1affb538,POINT (-5534176.250205638 9517775.873160409),MULTIPOLYGON (((-5534311.354149741 9517791.244...,15779,Trip hazards due to tree roots
6569,6989,cba2435c-ee48-46eb-bffe-27d2abbdd31c,POINT (-5534316.289239156 9517809.947056368),MULTIPOLYGON (((-5534372.383097034 9517915.681...,6216,Trip hazards due to tree roots
6570,6990,164102b2-6c84-4745-8eff-7f682af920db,POINT (-5534434.066349967 9517812.434087548),"POLYGON ((-5534394.238132634 9517799.69883411,...",5709,Trip hazards due to tree roots
6571,6991,47ee9771-854d-44bc-b268-7f0dace3ddb8,POINT (-5525989.1759943515 9517236.764555685),POLYGON ((-5525781.055860623 9517716.524368217...,22404,Fixed pinch point <36” sidewalk width


In [4]:
sidewalks_full = gpd.read_file('../data/shape/Sidewalks.shx')
sidewalks = gpd.GeoDataFrame(sidewalks_full)
sidewalks['ID'] = sidewalks.index

In [5]:
sidewalks_full.to_crs('3586')

,SWK_ID,MATERIAL,SWK_WIDTH,DISTRICT,SWK_AREA,PARENT,SEG_ID,SIDE,CG_ID,Rpr_Yr,New_SCI,Recon_Date,HPNETWORK,TEST_AREA,COST_RECON,GOODCANDID,TOTAREA,geometry,ID
0,1,CC,5.5,CHARLESTOWN,3648,ALBIO1,ALBIO1_0,LEFT,24842,0,85,NaN,0,0.000,0.0,0,3648.360,"POLYGON ((772436.303 2964399.527, 772449.080 2...",0
1,402,CC,6,SOUTH DORCHESTER,1671,ALMON1,ALMON1_1749,LEFT,401,0,55,NaN,1,1670.780,25061.7,0,1670.780,"POLYGON ((764945.142 2926518.871, 764942.704 2...",1
2,403,CC,6,SOUTH DORCHESTER,1351,ALMON1,ALMON1_2047,RIGHT,402,0,67,NaN,1,1351.330,20269.9,0,1351.330,"POLYGON ((764853.411 2926759.281, 764853.371 2...",2
3,404,CC,6,SOUTH DORCHESTER,1617,ALMON1,ALMON1_2047,LEFT,403,0,81,NaN,1,1616.860,24252.9,0,1616.860,"POLYGON ((764907.159 2926824.201, 764904.038 2...",3
4,405,CC,6,SOUTH DORCHESTER,3795,ALMON1,ALMON1_2338,RIGHT,404,2015,66,NaN,1,3795.180,56927.7,0,3795.180,"POLYGON ((764831.098 2927057.010, 764830.915 2...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23515,24087,CC,10,HYDE PARK,5589,CUMMI5,CUMMI5_2081,RIGHT,23878,0,0,NaN,1,2086.450,31296.8,0,2086.450,"POLYGON ((763550.688 2924072.918, 763558.222 2...",23515
23516,4547,CB,9,WEST ROXBURY,5609,CENTR6,CENTR6_25069,RIGHT,4534,0,84,NaN,1,2059.470,41189.4,0,2059.470,"MULTIPOLYGON (((750522.652 2929924.930, 750550...",23516
23517,21675,CC,6,WEST ROXBURY,1910,SPRIN7,SPRIN7_0,RIGHT,21538,0,85,NaN,0,0.000,0.0,0,722.677,"POLYGON ((747005.753 2926356.797, 747005.120 2...",23517
23518,5596,CC,4.5,NORTH END,1277,CHART1,CHART1_726,LEFT,5564,0,85,NaN,0,1277.020,19155.3,0,569.880,"POLYGON ((776379.139 2959259.142, 776370.530 2...",23518


In [6]:
grouped_hazard = sidewalks_hazard_score.groupby('ID')['ID'].count()
grouped_hazard = grouped_hazard.rename('hazard_score')
grouped_hazard = grouped_hazard.reset_index()

In [7]:
grouped_hazard

,ID,hazard_score
0,0,11
1,4,1
2,5,2
3,6,1
4,13,1
...,...,...
3701,23464,1
3702,23468,1
3703,23496,2
3704,23497,1


In [ ]:
grouped_hazard = pd.read_csv('../data/sidewalks_hazard_score_split.csv')
grouped_hazard['hazard_score'] = grouped_hazard['sum']

In [ ]:
grouped_ramp = sidewalks_ramp_score.groupby('ID')['ID'].count()
grouped_ramp = grouped_ramp.rename('ramp_score')
grouped_ramp = grouped_ramp.reset_index()

In [ ]:
sidewalks_score = pd.merge(grouped_ramp, grouped_hazard, on = 'ID',how = 'outer').fillna(0)
sidewalks_score['score'] = sidewalks_score['ramp_score'] + sidewalks_score['hazard_score']

In [ ]:
keep_columns = ['ID', 'Fixed pinch point <36” sidewalk width','Trip hazards due to tree roots',	'Trip hazards not due to tree roots', 'hazard_score', 'ramp_score', 'DISTRICT', 'score']

In [14]:
sidewalks_score =  pd.merge(sidewalks_score[keep_columns], sidewalks, on = 'ID',how = 'outer')[['ID', 'ramp_score', 'hazard_score', 'geometry', 'DISTRICT_x','score','Fixed pinch point <36” sidewalk width','Trip hazards due to tree roots',	'Trip hazards not due to tree roots']]

In [15]:
sidewalks_score = sidewalks_score[sidewalks_score['geometry']!=None]
sidewalks_score.fillna(0, inplace = True)

In [ ]:
gpd.GeoDataFrame(sidewalks_score).to_csv('../data/sidewalks_score_main.csv')

In [ ]:
sidewalks_score = pd.read_csv('../data/sidewalks_score_main.csv')

In [ ]:
from shapely.wkt import loads
sidewalks_score['geometry'] =  sidewalks_score['geometry'].apply(loads)

In [ ]:
import folium
import pandas as pd

# Load data
cnt = gpd.GeoDataFrame(sidewalks_score).set_crs('3586').to_crs(epsg=4326)
cnt = cnt.to_crs(epsg=4326)

cnt['cnt'] = cnt['score']
ramps_0 = pd.read_csv('../data/Ramps_0_with_score.csv')
sidewalks_hazards = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

sidewalks_hazards['x'] = sidewalks_hazards['x'].astype(float)
sidewalks_hazards['y'] = sidewalks_hazards['y'].astype(float)
map = folium.Map(location=[42.2995, -71.0649], zoom_start=15)

# Add sidewalks layer
# merged_sidewalk_hazards['centroids']= gpd.GeoSeries.from_wkt(merged_sidewalk_hazards['centroids'])
# merged_sidewalk_hazards = gpd.GeoDataFrame(merged_sidewalk_hazards)
# merged_sidewalk_hazards= merged_sidewalk_hazards.set_geometry('centroids')
# merged_sidewalk_hazards.crs = dest_epsg
# merged_sidewalk_hazards = merged_sidewalk_hazards.to_crs('4326')

def color_cluster_sidewalks(cnt_):
    # print(cnt_)
 
    cnt1 = cnt_['properties']['cnt']

    if cnt1 >  20:
        return  {'fillColor': '#fe1c37',  'color': '#fe1c37', 'fillOpacity' : '0.5'}
    elif cnt1 > 11:
        return  {'fillColor': '#db1a7a', 'color': '#db1a7a', 'fillOpacity' : '0.5'}
    elif cnt1 > 5:
        return  {'fillColor': '#704e9c', 'color': '#704e9c' ,'fillOpacity' : '0.5'}
    elif cnt1 > 3:
        return {'fillColor': '#254e6e', 'color': '#254e6e', 'fillOpacity' : '0.5'}
    else:
        return {'fillColor': '#015c30', 'color': '#015c30', 'fillOpacity' : '0.5'}
    
brown_layer = folium.FeatureGroup(name='Score more than 20 - Red')

red_layer = folium.FeatureGroup(name='Score more than 11 - Dark Pink')
orange_layer = folium.FeatureGroup(name = 'Score more than 5 - Violet')
yellow_layer = folium.FeatureGroup(name='Score more than 3 - Navy Blue')
green_layer = folium.FeatureGroup(name='Score 0 - Green')



def convert(row):
    t = shapely.geometry.mapping(row['geometry'])
    return {
       "type": "Feature",
  "geometry": {
    "type": row['geometry'].geom_type,
    "coordinates": t['coordinates']
  },
  "properties": {
    "cnt": row['cnt']
  }
    }
for index,row in cnt.iterrows():

    if row['geometry']== None:
        continue
    b = folium.GeoJson(convert(row), style_function = color_cluster_sidewalks)
    b.add_child(folium.Popup("Index                       : " + str(index) + "<br>Score: " + str(row['cnt'])
                         + "<br><br>Fixed pinch point     : " + str(row['Fixed pinch point <36” sidewalk width'])
                         + "<br><br>Trip due to tree roots: " + str(row['Trip hazards due to tree roots'])
                         + "<br><br>Trip due to cracks: " + str(row['Trip hazards not due to tree roots'])
                          + "<br><br>Bad Ramps: " + str(row['ramp_score']),
                         max_width=400))
    if row['cnt'] > 20:
        b.add_to(brown_layer)
    elif row['cnt'] > 11:
        b.add_to(red_layer)
    elif row['cnt'] > 5:
        b.add_to(orange_layer)
    elif row['cnt'] > 3:
        b.add_to(yellow_layer)
    else:
        b.add_to(green_layer)

brown_layer.add_to(map)
red_layer.add_to(map)
orange_layer.add_to(map)
yellow_layer.add_to(map)
green_layer.add_to(map)




folium.LayerControl().add_to(map)
map.save("../maps/sidewalks/sidewalks_with_score.html")